<a href="https://colab.research.google.com/github/mariafelecann/LicentaAI/blob/transformers/incercareLicenta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets --quiet

! pip install transformers==4.28.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
ERROR: pip's dependency resolver does not currently take into account all the packages th

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import json
import pandas as pd
import os
from datasets import load_dataset
from datasets import Dataset

import torch.nn.functional as F
from torch import nn
from transformers import AutoModel

from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from transformers import TrainerCallback, EarlyStoppingCallback
import csv


In [ ]:
CSV_FILE_PATH = "/content/romanian_music_filtered.csv"
RESULTS_PATH = "/content/results"
LOGS_PATH = "/content/logs"

In [ ]:
def read_raw_data(file_path):
    raw_data = []

    with open(file_path, encoding="utf-8") as file:
        reader = csv.DictReader(file, delimiter=",")  # Change to ',' if CSV is comma-separated
        for row in reader:
            item = {
                "title": row["title"],
                "tag": row["tag"],
                "artist": row["artist"],
                "lyrics": row["lyrics"],
                "id": row["id"]
            }
            raw_data.append(item)

    return raw_data


In [ ]:
import pandas as pd

df = pd.read_csv("/content/romanian_music_filtered.csv", delimiter=",", encoding="utf-8")  # Adjust delimiter if needed
print(df.columns)


Index(['title', 'tag', 'artist', 'lyrics', 'id'], dtype='object')


In [ ]:
print('Loading dataset...')
raw_data = read_raw_data(CSV_FILE_PATH)

# df = pd.DataFrame(raw_data)

# # Split dataset into train and test sets
# dataset = Dataset.from_pandas(df)
# hf_dataset_splits = dataset.train_test_split(test_size=0.3)

# train_dataset = hf_dataset_splits["train"]
# validation_dataset = hf_dataset_splits["test"]

# # Print dataset samples for verification
# print("Train dataset sample:", train_dataset[0])
# print("Validation dataset sample:", validation_dataset[0])
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import Dataset, DatasetDict

# Assuming your data has a 'genre' column
# First, create a balanced split for train/validation/test
df = pd.DataFrame(raw_data)
valid_genres = df['tag'].value_counts()[df['tag'].value_counts() >= 10].index

df_filtered = df[df['tag'].isin(valid_genres)]

X = df_filtered.drop('tag', axis=1)  # lyrics and other features
y = df_filtered['tag']

# First split: train + validation vs test (80% vs 20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y,
    test_size=0.2,  # 20% for test set
    stratify=y,     # Maintain genre balance
    random_state=42 # For reproducibility
)

# Second split: train vs validation (80% vs 20% of train_val)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=0.25,  # 20% of total data for validation
    stratify=y_train_val,
    random_state=42
)

# Create DataFrames for each split
df_train = pd.DataFrame(X_train).assign(tag=y_train)
df_val = pd.DataFrame(X_val).assign(tag=y_val)
df_test = pd.DataFrame(X_test).assign(tag=y_test)

train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})


Loading dataset...


In [ ]:
model_name = "dumitrescustefan/bert-base-romanian-cased-v1"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=7)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/397k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/500M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:442: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

In [ ]:
text = "Hello, how are you?"

# Create inputs
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

outputs = model(input_ids=input_ids, attention_mask=attention_mask)
print(outputs.keys())

odict_keys(['logits'])


In [ ]:

def preprocess_dataset(dataset, tokenizer):
    print("Available columns in dataset:", dataset.column_names)
    label2id = {label: i for i, label in enumerate(dataset.unique('tag'))}
    id2label = {i: label for i, label in enumerate(dataset.unique('tag'))}

    def tokenize_and_merge_title_content(examples):
        texts = [f"{line}" for line in examples['lyrics']]
        tokenized = tokenizer.batch_encode_plus(
            texts,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_token_type_ids=False,
        )
        return {
            "input_ids": tokenized["input_ids"],
            "attention_mask": tokenized["attention_mask"],
            # "label": examples["tag"]
             "label": [label2id[label] for label in examples["tag"]]  # Convert labels to integers
        }

    dataset = dataset.map(tokenize_and_merge_title_content, batched=True)
    dataset.set_format(
        type='torch',
        columns=['input_ids', 'attention_mask', 'label']
    )
    return dataset, label2id, id2label

In [ ]:
tokenized_train_dataset, label2id, id2label = preprocess_dataset(train_dataset, tokenizer)
tokenized_validation_dataset, _, _ = preprocess_dataset(val_dataset, tokenizer)


Available columns in dataset: ['title', 'artist', 'lyrics', 'id', 'tag', '__index_level_0__']


Map:   0%|          | 0/5304 [00:00<?, ? examples/s]

Available columns in dataset: ['title', 'artist', 'lyrics', 'id', 'tag', '__index_level_0__']


Map:   0%|          | 0/1768 [00:00<?, ? examples/s]

In [ ]:
class BestModelCheckpointCallback(TrainerCallback):
    def __init__(self, trainer, early_stopping_patience, output_dir):
        self.early_stopping_patience = early_stopping_patience
        self.best_val_acc = None
        self.patience_counter = 0
        self.output_dir = output_dir
        self.trainer = trainer

    def on_log(self, args, state, control, logs=None, **kwargs):
        val_acc = logs.get("eval_accuracy")
        if val_acc is None:
            return

        if self.best_val_acc is None or self.best_val_acc > val_acc:
            self.best_val_acc = val_acc
            self.patience_counter = 0
            self.save_best_model_checkpoint()
        else:
            self.patience_counter += 1
            if self.patience_counter >= self.early_stopping_patience:
                control.should_training_stop = True

    def save_best_model_checkpoint(self):
        if self.output_dir is not None:
            self.trainer.save_model(self.output_dir)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    loss = F.cross_entropy(torch.tensor(pred.predictions), torch.tensor(labels, dtype=torch.long)).item()
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'loss': loss
    }


training_args = TrainingArguments(
    output_dir=RESULTS_PATH,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=LOGS_PATH,
    learning_rate=2e-5,
    save_strategy="epoch",
    load_best_model_at_end=True,
    label_names=["label"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
CHECKPOINT_PATH = "/content/checkpoints"

In [ ]:
!wandb login


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mariafelecan19 (mariafelecan19-babes-bolyai-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)
# checkpoint_path = CHECKPOINT_PATH

# best_model_checkpoint_callback = BestModelCheckpointCallback(early_stopping_patience = 3,
#                                                              output_dir = checkpoint_path,
#                                                              trainer = trainer)

# trainer.add_callback(early_stopping_callback)
# trainer.add_callback(best_model_checkpoint_callback)

# trainer.train()
# Ensure callbacks are not duplicated before adding them
if not any(isinstance(cb, EarlyStoppingCallback) for cb in trainer.callback_handler.callbacks):
    early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)
    trainer.add_callback(early_stopping_callback)

if not any(isinstance(cb, BestModelCheckpointCallback) for cb in trainer.callback_handler.callbacks):
    checkpoint_path = CHECKPOINT_PATH
    best_model_checkpoint_callback = BestModelCheckpointCallback(
        early_stopping_patience=3,
        output_dir=checkpoint_path,
        trainer=trainer
    )
    trainer.add_callback(best_model_checkpoint_callback)

# Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.276200,No log


early stopping required metric_for_best_model, but did not find eval_loss so early stopping is disabled


KeyError: 'eval_loss'

In [ ]:
trainer_path = "/content/trainer"
trainer.save_model(trainer_path)


NameError: name 'trainer' is not defined

In [ ]:
trainer = BertForSequenceClassification.from_pretrained(trainer_path)

In [ ]:
def evaluate_custom_classifier(model, dataset, device):
    model.eval()

    correct_predictions = 0
    dataloader = DataLoader(dataset, batch_size=4)
    progress_bar = tqdm(dataloader, desc="Evaluate")

    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs, dim=1)
            correct_predictions += (preds == labels).sum().item()

    accuracy = correct_predictions / len(dataset)
    return accuracy


In [ ]:

def train_custom_classifier(model, train_dataset, val_dataset, optimizer, device, num_epochs):
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        epoch_loss = 0
        correct_predictions = 0

        train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
        progress_bar = tqdm(train_dataloader, desc="Train")

        for batch in progress_bar:
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = torch.nn.functional.cross_entropy(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            correct_predictions += (preds == labels).sum().item()

        epoch_loss /= len(train_dataset)
        accuracy = correct_predictions / len(train_dataset)

        print(f"Train Loss: {epoch_loss:.5f}, Train Accuracy: {accuracy:.5f}")
        val_accuracy = evaluate_custom_classifier(model, val_dataset, device)
        print(f"Validation Accuracy: {val_accuracy:.5f}")

In [ ]:
class CustomClassifier(nn.Module):
    def __init__(self, pretrained_model, num_classes):
        super(CustomClassifier, self).__init__()
        self.base_model = pretrained_model
        self.dropout = nn.Dropout(0.2)
        self.dense = nn.Linear(self.base_model.config.hidden_size, 128)
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state
        x = hidden_state[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.relu(x)
        x = self.classifier(x)
        return x

    def predict(self, input_ids, attention_mask):
        outputs = self.forward(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs, dim=1)
        return predictions

In [ ]:
custom_classifier = CustomClassifier(trainer.base_model, 2)

optimizer = torch.optim.AdamW(custom_classifier.parameters(), lr=2e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_custom_classifier(custom_classifier, tokenized_train_dataset, tokenized_validation_dataset, optimizer, device, num_epochs=5)


In [ ]:
custom_classifier_path ="/content/custom_classifier"
torch.save(custom_classifier.state_dict(), custom_classifier_path)


In [ ]:
classifier_model = CustomClassifier(trainer.base_model, 2)
classifier_model.load_state_dict(torch.load(custom_classifier_path, map_location=torch.device('cpu')))
classifier_model.to(device)

In [ ]:
tokenized_test_dataset = preprocess_dataset(test_dataset, tokenizer)
test_accuracy = evaluate_custom_classifier(custom_classifier, tokenized_test_dataset, device)

In [ ]:
def predict_labels(model, dataset, device):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=8)
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            outputs = model.predict(input_ids=input_ids, attention_mask=attention_mask)
            predictions.extend(outputs.cpu().numpy())

    return predictions

In [ ]:
predictions = predict_labels(custom_classifier, tokenized_test_dataset, device)
true_labels = tokenized_test_dataset["labels"]